In [1]:
import pyaudio

p = pyaudio.PyAudio()

info = p.get_host_api_info_by_index(0)
num_devices = info.get('deviceCount')

for i in range(0, num_devices):
    device_info = p.get_device_info_by_host_api_device_index(0, i)
    print(f"Device {i}: {device_info['name']}")
    print(f"Supported sample rates:")
    for rate in [8000, 16000, 22050, 44100, 48000]:
        try:
            if p.is_format_supported(rate,
                                     input_device=device_info['index'],
                                     input_channels=1,
                                     input_format=pyaudio.paInt16):
                print(f"  {rate} Hz")
        except ValueError:
            pass

p.terminate()


Device 0: Microsoft Sound Mapper - Input
Supported sample rates:
  8000 Hz
  16000 Hz
  22050 Hz
  44100 Hz
  48000 Hz
Device 1: 마이크(2- MATA STUDIO C10)
Supported sample rates:
  8000 Hz
  16000 Hz
  22050 Hz
  44100 Hz
  48000 Hz
Device 2: Britz 마이크(Britz BR-MICBAR)
Supported sample rates:
  8000 Hz
  16000 Hz
  22050 Hz
  44100 Hz
  48000 Hz
Device 3: Microsoft Sound Mapper - Output
Supported sample rates:
Device 4: Britz 스피커(Britz BR-MICBAR)
Supported sample rates:
Device 5: Digital Audio (S/PDIF)(High Def
Supported sample rates:
Device 6: 헤드폰(2- MATA STUDIO C10)
Supported sample rates:
Device 7: 모니터 스피커(NVIDIA High Definition 
Supported sample rates:


In [2]:
import sounddevice as sd
import numpy as np
import librosa
import tensorflow as tf
import time
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtWidgets import QApplication, QWidget
from PyQt5.QtGui import QIcon
import webrtcvad
import pyaudio
import wave
import noisereduce as nr
from faster_whisper import WhisperModel
import datetime
import os
import sys

# 현재 작업 디렉토리를 기준으로 모델 경로 설정 
model_path = os.path.join(os.getcwd(), 'my_model.h5')

# 학습된 모델 로드
model = tf.keras.models.load_model(model_path)


# UrbanSound8K의 분류 라벨들
class_labels = ['air_conditioner', 'car_horn', 'children_playing', 'dog_bark', 'drilling',
                'engine_idling', 'gun_shot', 'jackhammer', 'siren', 'street_music']

# 오디오 데이터 전처리 함수 (MFCC로 변환)
def preprocess_audio(audio, sample_rate=22050):
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)

    # Conv2D 레이어에 맞게 크기를 40x174로 패딩
    if mfccs.shape[1] < 174:  # 프레임 수가 174보다 작으면 패딩
        mfccs = np.pad(mfccs, ((0, 0), (0, 174 - mfccs.shape[1])), mode='constant')
    
    return mfccs

class STTWorker(QtCore.QThread):
    text_update = QtCore.pyqtSignal(str)
    stt_finished = QtCore.pyqtSignal(str)

    def __init__(self, model_size="small"):
        super(STTWorker, self).__init__()
        self.vad = webrtcvad.Vad(3)
        self.model = WhisperModel(model_size, device="cpu")
        self.channels = 1
        self.frame_duration = 20
        self.frame_size = None
        self.running = False
        self.frames = []

        self.p = pyaudio.PyAudio()
        self.sample_rate = self.get_supported_sample_rate()
        self.frame_size = int(self.sample_rate * self.frame_duration / 1000)

        self.stream = self.p.open(format=pyaudio.paInt16, channels=self.channels,
                                  rate=self.sample_rate, input=True,
                                  frames_per_buffer=self.frame_size)

    def get_supported_sample_rate(self):
        # 확인할 샘플 레이트 목록
        sample_rates = [16000, 22050, 44100, 48000]
        default_sample_rate = 16000  # 기본 샘플 레이트

        # 입력 장치 정보 확인
        info = self.p.get_host_api_info_by_index(0)
        num_devices = info.get('deviceCount')

        # 각 입력 장치의 지원 샘플 레이트 확인
        for i in range(0, num_devices):
            device_info = self.p.get_device_info_by_host_api_device_index(0, i)
            for rate in sample_rates:
                try:
                    if self.p.is_format_supported(rate,
                                                  input_device=device_info['index'],
                                                  input_channels=self.channels,
                                                  input_format=pyaudio.paInt16):
                        print(f"장치 {device_info['name']}에서 {rate} Hz 샘플레이트 지원됨")
                        return rate  # 첫 번째로 지원되는 샘플 레이트를 반환
                except ValueError:
                    pass
        
        print(f"지원되는 샘플레이트를 찾을 수 없음, 기본값 {default_sample_rate} Hz 사용")
        return default_sample_rate  # 지원되는 샘플 레이트가 없을 경우 기본값 반환

    def run(self):
        self.running = True
        self.frames = []
        while self.running:
            audio_frame = self.stream.read(self.frame_size)
            self.frames.append(audio_frame)

    def stop(self):
        self.running = False
        self.process_audio()

    def process_audio(self):
        audio_data = b''.join(self.frames)
        clean_segment = self.reduce_noise(audio_data)
        stt_result = self.transcribe_audio(clean_segment)
        for text_segment in stt_result:
            self.stt_finished.emit(text_segment.text)

    def reduce_noise(self, audio_data):
        audio_array = np.frombuffer(audio_data, dtype=np.int16).astype(np.float32)
        reduced_noise = nr.reduce_noise(y=audio_array, sr=self.sample_rate)
        return reduced_noise.astype(np.int16).tobytes()

    def transcribe_audio(self, audio_data):
        audio_array = np.frombuffer(audio_data, dtype=np.int16).astype(np.float32) / 32768.0
        segments, _ = self.model.transcribe(audio_array, language='ko')
        return segments


class SoundClassifierWorker(QtCore.QThread):
    sound_classified = QtCore.pyqtSignal(str)  # Signal to send classified sound text

    def __init__(self, parent=None):
        super(SoundClassifierWorker, self).__init__(parent)
        self.running = True  # 스레드의 동작 상태를 제어하기 위한 플래그

    def run(self):
        while self.running:
            self.classify_sound()  # 소리 분석
            time.sleep(2)  # N초 대기 후 다시 분석 (필요에 따라 조정 가능)

    def classify_sound(self):
        # 샘플링 레이트 및 녹음 시간
        sample_rate = 22050
        duration = 2  # 2초 동안 녹음

        # 마이크로부터 음성 녹음
        audio = sd.rec(int(sample_rate * duration), samplerate=sample_rate, channels=1)
        sd.wait()  

        # 음성 데이터를 전처리하여 모델 입력 형식으로 변환
        audio = audio.flatten()  # 2D 배열을 1D로 변환
        mfccs = preprocess_audio(audio, sample_rate)

        # MFCC 데이터를 (batch_size, height, width, channels) 형식으로 변환
        mfccs = np.expand_dims(mfccs, axis=-1)  
        mfccs = np.expand_dims(mfccs, axis=0)   

        # 모델로 예측 수행
        prediction = model.predict(mfccs)

        # 각 클래스에 대한 예측 확률을 백분율로 변환
        prediction_percentage = prediction[0] * 100  

        # 현재 시간 가져오기
        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        result_text = ""

        # 50% 이상의 확률을 가진 클래스만 출력
        for i, label in enumerate(class_labels):
            if prediction_percentage[i] > 50:  
                result_text += f"{label}: {prediction_percentage[i]:.2f}% ({current_time})\n"

        # 가장 높은 확률의 클래스를 예측 결과로 출력
        predicted_label = np.argmax(prediction)
        if prediction_percentage[predicted_label] < 50:  # 가장 높은 확률이 50% 이하일 때만 출력
             result_text += "No sound detected with more than 50% confidence.\n"
       

        # 결과를 signal로 전송
        self.sound_classified.emit(result_text)

    def stop(self):
        self.running = False  


class Ui_STT(object):
    def setupUi(self, STT):
        STT.setObjectName("STT")
        STT.resize(800, 480)  # 창 크기를 더 키움
        STT.setCursor(QtGui.QCursor(QtCore.Qt.ArrowCursor))

        # TextEdit
        self.textEdit = QtWidgets.QTextEdit(STT)
        self.textEdit.setGeometry(QtCore.QRect(20, 20, 760, 250))  # 창 크기에 맞게 TextEdit 조정
        self.textEdit.setObjectName("textEdit")
        
        # Buttons
        self.pushButton = QtWidgets.QPushButton(STT)
        self.pushButton.setGeometry(QtCore.QRect(20, 300, 150, 70))
        self.pushButton.setObjectName("pushButton")
        
        self.stopButton = QtWidgets.QPushButton(STT)  # '모든 작업 종료' 버튼
        self.stopButton.setGeometry(QtCore.QRect(20, 380, 150, 70))  # '녹음' 버튼 밑으로 이동
        self.stopButton.setObjectName("stopButton")
        
        self.pushButton_2 = QtWidgets.QPushButton(STT)
        self.pushButton_2.setGeometry(QtCore.QRect(180, 300, 150, 70))
        self.pushButton_2.setObjectName("pushButton_2")
        
        self.pushButton_3 = QtWidgets.QPushButton(STT)
        self.pushButton_3.setGeometry(QtCore.QRect(180, 380, 150, 70))
        self.pushButton_3.setObjectName("pushButton_3")

        self.pushButton_small = QtWidgets.QPushButton(STT)
        self.pushButton_small.setGeometry(QtCore.QRect(340, 300, 150, 70))
        self.pushButton_small.setObjectName("pushButton_small")
        
        self.pushButton_medium = QtWidgets.QPushButton(STT)
        self.pushButton_medium.setGeometry(QtCore.QRect(340, 380, 150, 70))
        self.pushButton_medium.setObjectName("pushButton_medium")

        self.newButton = QtWidgets.QPushButton(STT)  
        self.newButton.setGeometry(QtCore.QRect(500, 300, 150, 70))
        self.newButton.setObjectName("newButton")

        self.retranslateUi(STT)
        QtCore.QMetaObject.connectSlotsByName(STT)

        # Connect buttons to their functions
        self.pushButton.clicked.connect(self.start_recording)
        self.pushButton_2.clicked.connect(self.stop_recording)
        self.pushButton_3.clicked.connect(self.clear_text)
        self.pushButton_small.clicked.connect(self.set_small_model)
        self.pushButton_medium.clicked.connect(self.set_medium_model)
        self.newButton.clicked.connect(self.start_classification)  # 소리 분석 버튼
        self.stopButton.clicked.connect(self.stop_all)  # 모든 작업 중지 버튼

        self.classifier_worker = None  # 분석 스레드
        self.stt_worker = None  # STT 스레드
        self.model_size = "small"  # 기본 모델 크기를 small로 설정
        self.mode = None  # 현재 모드 ('STT' 또는 'classification')

        # 창 가장자리 스타일 적용
        STT.setStyleSheet("""
            QWidget {
                border: 2px solid gray;
                border-radius: 10px;
                padding: 5px;
            }
        """)

        # 녹음 버튼 (가장자리와 글자색만 빨간색)
        self.pushButton.setStyleSheet("""
            QPushButton {
                color: red;
                border: 2px solid red;
                border-radius: 10px;
                background-color: transparent;
                padding: 10px;
            }
            QPushButton:hover {
                background-color: #ffcccc;
            }
        """)

        # 종료 버튼 (파란색)
        self.pushButton_2.setStyleSheet("""
            QPushButton {
                color: blue;
                border: 2px solid blue;
                border-radius: 10px;
                background-color: transparent;
                padding: 10px;
            }
            QPushButton:hover {
                background-color: #cceeff;
            }
        """)

        # 소리 분석 버튼 (주황색)
        self.newButton.setStyleSheet("""
            QPushButton {
                color: orange;
                border: 2px solid orange;
                border-radius: 10px;
                background-color: transparent;
                padding: 10px;
            }
            QPushButton:hover {
                background-color: #ffcc99;
            }
        """)

        # 출력 초기화 버튼 (회색)
        self.pushButton_3.setStyleSheet("""
            QPushButton {
                color: gray;
                border: 2px solid gray;
                border-radius: 10px;
                background-color: transparent;
                padding: 10px;
            }
            QPushButton:hover {
                background-color: #e0e0e0;
            }
        """)

        # small 버튼 (마우스 오버 시 옅은 회색)
        self.pushButton_small.setStyleSheet("""
            QPushButton {
                color: black;
                border: 2px solid black;
                border-radius: 10px;
                background-color: transparent;
                padding: 10px;
            }
            QPushButton:hover {
                background-color: #e0e0e0;
            }
        """)

        # medium 버튼 (마우스 오버 시 옅은 회색)
        self.pushButton_medium.setStyleSheet("""
            QPushButton {
                color: black;
                border: 2px solid black;
                border-radius: 10px;
                background-color: transparent;
                padding: 10px;
            }
            QPushButton:hover {
                background-color: #e0e0e0;
            }
        """)

        # 모든 작업 중지 버튼 (마우스 오버 시 옅은 회색)
        self.stopButton.setStyleSheet("""
            QPushButton {
                color: black;
                border: 2px solid black;
                border-radius: 10px;
                background-color: transparent;
                padding: 10px;
            }
            QPushButton:hover {
                background-color: #e0e0e0;
            }
        """)
    # 아이콘 설정 (setWindowIcon 호출)
        icon_path = os.path.join(os.getcwd(), 'resources', 'free-icon-font-play-3917517.png')
        if os.path.exists(icon_path):
            STT.setWindowIcon(QtGui.QIcon(icon_path))
        else:
            print(f"아이콘 경로를 찾을 수 없습니다: {icon_path}")

    def retranslateUi(self, STT):
        _translate = QtCore.QCoreApplication.translate
        STT.setWindowTitle(_translate("STT", "PLAY"))
        self.pushButton.setText(_translate("STT", "녹음"))
        self.pushButton_2.setText(_translate("STT", "종료"))
        self.pushButton_3.setText(_translate("STT", "출력 초기화"))
        self.pushButton_small.setText(_translate("STT", "small"))
        self.pushButton_medium.setText(_translate("STT", "medium"))
        self.newButton.setText(_translate("STT", "소리 분석"))
        self.stopButton.setText(_translate("STT", "모든 작업 중지"))

    # New button function to start sound classification
    def start_classification(self):
        if self.mode == "STT":  # STT 모드 실행 중이라면 종료
            self.stop_recording()

        if self.classifier_worker is None or not self.classifier_worker.isRunning():
            self.textEdit.append("소리 분석을 시작합니다...")
            self.classifier_worker = SoundClassifierWorker()
            self.classifier_worker.sound_classified.connect(self.update_text)
            self.classifier_worker.start()
            self.mode = "classification"  # 소리 분석 모드로 설정

    def start_recording(self):
        if self.mode == "classification":  # 소리 분석 모드 실행 중이라면 종료
            self.stop_classification()

        if self.stt_worker is None or not self.stt_worker.isRunning():
            self.textEdit.append(f"녹음을 시작합니다... (모델: {self.model_size})")
            self.stt_worker = STTWorker(model_size=self.model_size)
            self.stt_worker.text_update.connect(self.update_text)
            self.stt_worker.stt_finished.connect(self.update_text)
            self.stt_worker.start()
            self.mode = "STT"  # STT 모드로 설정

    def stop_recording(self):
        if self.stt_worker is not None and self.stt_worker.isRunning():
            self.stt_worker.stop()
            self.stt_worker = None
            self.mode = None  # STT 모드 종료

    def stop_classification(self):
        if self.classifier_worker is not None:
            self.classifier_worker.stop()
            self.classifier_worker = None
            self.textEdit.append("소리 분석이 중지되었습니다.")
            self.mode = None  # 소리 분석 모드 종료

    def stop_all(self):
        if self.mode == "STT":
            self.stop_recording()
        elif self.mode == "classification":
            self.stop_classification()

    def clear_text(self):
        self.textEdit.clear()

    def set_small_model(self):
        self.model_size = "small"
        self.textEdit.append("모델이 'small'로 설정되었습니다.")

    def set_medium_model(self):
        self.model_size = "medium"
        self.textEdit.append("모델이 'medium'로 설정되었습니다.")

    def update_text(self, text):
        self.textEdit.append(f"{text}")

if __name__ == "__main__":
    import sys
    app = QtWidgets.QApplication(sys.argv)
    STT = QtWidgets.QWidget()
    ui = Ui_STT()
    ui.setupUi(STT)
    STT.show()
    sys.exit(app.exec_())

c:\Anaconda\envs\home\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


장치 Microsoft Sound Mapper - Input에서 16000 Hz 샘플레이트 지원됨


SystemExit: 0

c:\Anaconda\envs\home\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
